In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t
import math

In [3]:
spark = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .enableHiveSupport() \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
    .getOrCreate()

"""
Reference gresearch:
- Parquet files analysis: https://www.gresearch.com/blog/article/parquet-files-know-your-scaling-limits/
- GitHub Spark extension: https://github.com/G-Research/spark-extension
- Parquet methods: https://github.com/G-Research/spark-extension/tree/master/python/gresearch/spark/parquet
"""

'\nReference gresearch:\n- Parquet files analysis: https://www.gresearch.com/blog/article/parquet-files-know-your-scaling-limits/\n- GitHub Spark extension: https://github.com/G-Research/spark-extension\n- Parquet methods: https://github.com/G-Research/spark-extension/tree/master/python/gresearch/spark/parquet\n'

In [7]:
sc = spark.sparkContext

In [8]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")
#to not cache dataframes... this may not create repeatable results
spark.conf.set("spark.databricks.io.cache.enabled", "false")

In [9]:
def sdf_generator(num_rows: int, num_partitions: int = None) -> "DataFrame":
    return (
        spark.range(num_rows, numPartitions=num_partitions)
        .withColumn("date", f.current_date())
        .withColumn("timestamp",f.current_timestamp())
        .withColumn("idstring", f.col("id").cast("string"))
        .withColumn("idfirst", f.col("idstring").substr(0,1))
        .withColumn("idlast", f.col("idstring").substr(-1,1))
        )

In [10]:
sdf = sdf_generator(10000000, 8)
sdf.show()

+---+----------+--------------------+--------+-------+------+
| id|      date|           timestamp|idstring|idfirst|idlast|
+---+----------+--------------------+--------+-------+------+
|  0|2024-03-15|2024-03-15 06:32:...|       0|      0|     0|
|  1|2024-03-15|2024-03-15 06:32:...|       1|      1|     1|
|  2|2024-03-15|2024-03-15 06:32:...|       2|      2|     2|
|  3|2024-03-15|2024-03-15 06:32:...|       3|      3|     3|
|  4|2024-03-15|2024-03-15 06:32:...|       4|      4|     4|
|  5|2024-03-15|2024-03-15 06:32:...|       5|      5|     5|
|  6|2024-03-15|2024-03-15 06:32:...|       6|      6|     6|
|  7|2024-03-15|2024-03-15 06:32:...|       7|      7|     7|
|  8|2024-03-15|2024-03-15 06:32:...|       8|      8|     8|
|  9|2024-03-15|2024-03-15 06:32:...|       9|      9|     9|
| 10|2024-03-15|2024-03-15 06:32:...|      10|      1|     0|
| 11|2024-03-15|2024-03-15 06:32:...|      11|      1|     1|
| 12|2024-03-15|2024-03-15 06:32:...|      12|      1|     2|
| 13|202

In [11]:
sdf_schema = "id bigint, date date, timestamp timestamp, idstring string, idfirst string, idlast string"

How predicate pushdown works on Spark since Spark 3.1.0: https://www.waitingforcode.com/apache-spark-sql/what-new-apache-spark-3.1-predicate-pushdown-json-csv-apache-avro/read

# 1 - JSON

## 1-1 Initial state JSON

How it works: https://github.com/jerryshao/apache-spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/DataFrameReader.scala#L323
https://spark.apache.org/docs/latest/sql-data-sources-json.html

In [8]:
sc.setJobDescription("Save Json")
path_json = "D:/Spark/Data/format_json_large.json"
sdf.write.format("json").mode("overwrite").save(path_json)

In [9]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 160
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

In [10]:
sc.setJobDescription("Load Json all data")
sdf_json = spark.read.format("json").schema(sdf_schema).load(path_json)
sdf_json.write.format("noop").mode("overwrite").save()

## 1-2 Column filter

In [11]:
sc.setJobDescription("JSON Column Filter")
sdf_json = spark.read.format("json").schema(sdf_schema).load(path_json)
sdf_json = sdf_json.select("id", "idstring")
sdf_json.write.format("noop").mode("overwrite").save()

## 1-3 Row filter

In [12]:
sc.setJobDescription("JSON Row Filter id")
sdf_json = spark.read.format("json").schema(sdf_schema).load(path_json)
sdf_json = sdf_json.filter(f.col("id") < 300)
sdf_json.write.format("noop").mode("overwrite").save()

# 2 - CSV with schema interference

## 2-1 Initial state CSV

How it works: https://github.com/jerryshao/apache-spark/blob/master/sql/core/src/main/scala/org/apache/spark/sql/DataFrameReader.scala#L470
https://spark.apache.org/docs/latest/sql-data-sources-csv.html

In [13]:
sc.setJobDescription("Save CSV")
path_csv = "D:/Spark/Data/format_csv_large.csv"
sdf.write.format("csv").mode("overwrite").option("header", "True").save(path_csv)

In [14]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 80
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

In [15]:
sc.setJobDescription("Load CSV All data")
sdf_csv = spark.read.format("csv").options(header=True).schema(sdf_schema).load(path_csv)
sdf_csv.write.format("noop").mode("overwrite").save()

## 2-2 Column Filter

In [16]:
sc.setJobDescription("CSV Column Filter")
sdf_csv = spark.read.format("csv").schema(sdf_schema).load(path_csv)
sdf_csv = sdf_csv.select("id", "idstring")
sdf_csv.write.format("noop").mode("overwrite").save()

## 2-3 Row Filter

In [17]:
sc.setJobDescription("CSV Row Filter id")
sdf_csv = spark.read.format("csv").schema(sdf_schema).load(path_csv)
sdf_csv = sdf_csv.filter(f.col("id") < 300)
sdf_csv.write.format("noop").mode("overwrite").save()

# 3 - Parquet

## 3-1 Initial state Parquet

How it works: https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

In [55]:
sc.setJobDescription("Save Parquet")
path_parquet = "D:/Spark/Data/format_parquet_large.parquet"
sdf.write.format("parquet").mode("overwrite").save(path_parquet)

In [13]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 15
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

Introducing Spark config: spark.sql.sources.useV1SourceList, default: avro,csv,json,kafka,orc,parquet,text

A comma-separated list of data source short names or fully qualified data source 
implementation class names for which Data Source V2 code path is disabled. These 
data sources will fallback to Data Source V1 code path.

Data Source V2 is a more flexible data source class supporting easier implementation of new pushdown options and also might be a bit more efficient.

https://www.youtube.com/watch?v=U5n-evWfYSQ

https://github.com/apache/spark/blob/fe0aa1edff047689a1906eaa86d667e0ddfabfbc/sql/catalyst/src/main/scala/org/apache/spark/sql/internal/SQLConf.scala#L3207

In [14]:
#TODO: Check with Spark community why V2 not activated for parquet and aggregate not pushed as default
#TODO: Why statistics eliminated from Scanning Tab... Inefficiency?
#TODO: filter and count does not work

In [15]:
spark.conf.set("spark.sql.sources.useV1SourceList", "parquet")
sc.setJobDescription("Load Parquet all data V1")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet.write.format("noop").mode("overwrite").save()

In [16]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
sc.setJobDescription("Load Parquet all data V2")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet.write.format("noop").mode("overwrite").save()

## 3-2 Column Filter

In [31]:
spark.conf.set("spark.sql.sources.useV1SourceList", "parquet")
sc.setJobDescription("Parquet Column Filter V1")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet = sdf_parquet.select("id", "idstring")
sdf_parquet.write.format("noop").mode("overwrite").save()

In [32]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
sc.setJobDescription("Parquet Column Filter V2")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet = sdf_parquet.select("id", "idstring")
sdf_parquet.write.format("noop").mode("overwrite").save()

## 3-3 Row Filter id

In [23]:
spark.conf.set("spark.sql.sources.useV1SourceList", "parquet")
sc.setJobDescription("Parquet Row Filter id V1")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet = sdf_parquet.filter(f.col("id") < 300)
sdf_parquet.write.format("noop").mode("overwrite").save()

In [24]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
sc.setJobDescription("Parquet Row Filter id V2")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet = sdf_parquet.filter(f.col("id") < 300)
sdf_parquet.write.format("noop").mode("overwrite").save()

## 3-4 Count

In [33]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
spark.conf.set("spark.sql.parquet.aggregatePushdown", "false")
sc.setJobDescription("Parquet count aggegratePushdown false")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet.count()

10000000

Introducing: spark.sql.parquet.aggregatePushdown, default false, needs data source not to be listed in spark.sql.sources.useV1SourceList

If true, aggregates will be pushed down to Parquet for optimization. Support MIN, MAX and COUNT as aggregate expression. For MIN/MAX, support boolean, integer, float and date type. For COUNT, support all data types. If statistics is missing from any Parquet file footer, exception would be thrown.

https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

In [35]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
spark.conf.set("spark.sql.parquet.aggregatePushdown", "true")
sc.setJobDescription("Parquet count aggegratePushdown true")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet.count()

10000000

## 3-5 Max

In [37]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
spark.conf.set("spark.sql.parquet.aggregatePushdown", "false")
sc.setJobDescription("Parquet max aggegratePushdown false")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_max = sdf_parquet.groupBy().max("id")
sdf_max.show()

+-------+
|max(id)|
+-------+
|9999999|
+-------+



In [36]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
spark.conf.set("spark.sql.parquet.aggregatePushdown", "true")
sc.setJobDescription("Parquet max aggegratePushdown true")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_max = sdf_parquet.groupBy().max("id")
sdf_max.show()

+-------+
|max(id)|
+-------+
|9999999|
+-------+



## 3-6 filter and a count

In [40]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
spark.conf.set("spark.sql.parquet.aggregatePushdown", "true")
sc.setJobDescription("Parquet count with filter 1250000")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet = sdf_parquet.filter(f.col("id") < 1250000)
sdf_parquet.count()

1250000

## 3-7 filter and max

In [57]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
spark.conf.set("spark.sql.parquet.aggregatePushdown", "true")
sc.setJobDescription("Parquet max with filter 1250000")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet)
sdf_parquet = sdf_parquet.filter(f.col("id") <= 1249999)
sdf_max = sdf_parquet.groupBy().max("id")
sdf_max.show()

+-------+
|max(id)|
+-------+
|1249999|
+-------+



## 3-8 Partitioned and filter

In [51]:
sc.setJobDescription("Save Parquet Partitioned")
path_parquet_part = "D:/Spark/Data/format_parquet_partitioned.parquet"
sdf.repartition("idlast").write.partitionBy("idlast").format("parquet").mode("overwrite").save(path_parquet)

In [52]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
sc.setJobDescription("Parquet Row Filter idlast Partitioned")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet_part)
sdf_parquet = sdf_parquet.filter(f.col("idlast") == "1")
sdf_parquet.write.format("noop").mode("overwrite").save()

## 3-9 Partitioned, filter and max

In [53]:
spark.conf.set("spark.sql.sources.useV1SourceList", "")
spark.conf.set("spark.sql.parquet.aggregatePushdown", "true")
sc.setJobDescription("Parquet partitioned max with filter idlast 1")
sdf_parquet = spark.read.format("parquet").schema(sdf_schema).load(path_parquet_part)
sdf_parquet = sdf_parquet.filter(f.col("idlast") == "1")
sdf_max = sdf_parquet.groupBy().max("id")
sdf_max.show()

+-------+
|max(id)|
+-------+
|9999991|
+-------+



# 4 - Avro

## 4-1 Initial state AVRO

How it works: https://spark.apache.org/docs/latest/sql-data-sources-avro.html

In [18]:
sc.setJobDescription("Save Avro")
path_avro = "D:/Spark/Data/format_avro_large.avro"
sdf.write.format("avro").mode("overwrite").save(path_avro)

In [19]:
#set maxPartitions MB for a fair comparison of 8 partitions also during read 
maxPartitionsMB = 10
maxPartitionsBytes = math.ceil(maxPartitionsMB*1024*1024)
spark.conf.set("spark.sql.files.maxPartitionBytes", str(maxPartitionsBytes)+"b")

In [20]:
sc.setJobDescription("Load Avro All data")
sdf_avro = spark.read.format("avro").schema(sdf_schema).load(path_avro)
sdf_avro.write.format("noop").mode("overwrite").save()

## 4-2 Column Filter

Results:
- Load data: 1.6 s

In [21]:
sc.setJobDescription("Avro Column Filter")
sdf_avro = spark.read.format("avro").schema(sdf_schema).load(path_avro)
sdf_avro = sdf_avro.select("id", "idstring")
sdf_avro.write.format("noop").mode("overwrite").save()

## 4-3 Row Filter

In [22]:
sc.setJobDescription("Avro ID Filter")
sdf_avro = spark.read.format("avro").schema(sdf_schema).load(path_avro)
sdf_avro = sdf_avro.filter(f.col("id") < 300)
sdf_avro.write.format("noop").mode("overwrite").save()